In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [2]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float', 
'usosmultiples': 'float', 
'piscina': 'float', 
'escuelascercanas': 'float', 
'centroscomercialescercanos': 'float', 
'precio': 'float', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [3]:
data["ciudad"].nunique()

875

In [4]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [5]:
data.isnull().sum().sum()

535127

In [6]:
data.size

5520000

In [7]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# Levanto el csv de test para calcularle los features en paralelo

In [8]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [9]:
len(test)

60000

# Se completan NULLs con extensión de valores válidos

Lo que se hace es ordenar el dataset por tipodepropiedad. A continuacion se llama a la funcion fillna con el metodo ffill, que propaga la ultima observacion valida hasta la nueva observacion valida. De esta manera deberiamos llenar de forma mas o menos correcta. En caso de que haya quedado algun null, utilizo bfill

## Ordeno el dataset

In [10]:
data = data.sort_values(by = 'tipodepropiedad')

In [11]:
test = test.sort_values(by = 'tipodepropiedad')

## Metroscubiertos

In [12]:
data[['metroscubiertos']] = data[['metroscubiertos']].fillna(method = "ffill")

In [13]:
data["metroscubiertos"].isnull().sum().sum()

0

In [14]:
test[['metroscubiertos']] = test[['metroscubiertos']].fillna(method = "ffill")

In [15]:
test["metroscubiertos"].isnull().sum().sum()

1

In [16]:
test['metroscubiertos'] = test['metroscubiertos'].fillna(method = "bfill")

In [17]:
test["metroscubiertos"].isnull().sum().sum()

0

## Metrostotales

In [18]:
data['metrostotales'] = data['metrostotales'].fillna(method = "ffill")

In [19]:
data['metrostotales'].isnull().sum()

0

In [20]:
test['metrostotales'] = test['metrostotales'].fillna(method = "ffill")

In [21]:
test['metrostotales'].isnull().sum()

0

## Antiguedad

In [22]:
data['antiguedad'] = data['antiguedad'].fillna(method = "ffill")

In [23]:
data["antiguedad"].isnull().sum().sum()

14

In [24]:
data['antiguedad'] = data['antiguedad'].fillna(method = "bfill")

In [25]:
data["antiguedad"].isnull().sum().sum()

0

In [26]:
test['antiguedad'] = test['antiguedad'].fillna(method = "ffill")

In [27]:
test["antiguedad"].isnull().sum().sum()

0

## Banos

In [28]:
data['banos'] = data['banos'].fillna(method = "ffill")

In [29]:
data["banos"].isnull().sum().sum()

0

In [30]:
test['banos'] = test['banos'].fillna(method = "ffill")

In [31]:
test["banos"].isnull().sum().sum()

1

In [32]:
test['banos'] = test['banos'].fillna(method = "bfill")

In [33]:
test["banos"].isnull().sum().sum()

0

## Garages

In [34]:
data['garages'] = data['garages'].fillna(method = "ffill")

In [35]:
data["garages"].isnull().sum().sum()

0

In [36]:
test['garages'] = test['garages'].fillna(method = "ffill")

In [37]:
test["garages"].isnull().sum().sum()

3

In [38]:
test['garages'] = test['garages'].fillna(method = "bfill")

In [39]:
test["garages"].isnull().sum().sum()

0

## Re ordeno el dataset segun ciudad para fillear latitud y longitud (que al parecer es muy importante)

In [40]:
data = data.sort_values(by = 'ciudad')

In [41]:
test = test.sort_values(by = 'ciudad')

## Latitud

In [42]:
data['lat'] = data['lat'].fillna(method = "ffill")

In [43]:
data["lat"].isnull().sum().sum()

2

In [44]:
data['lat'] = data['lat'].fillna(method = "bfill")

In [45]:
data["lat"].isnull().sum().sum()

0

In [46]:
test['lat'] = test['lat'].fillna(method = "ffill")

In [47]:
test["lat"].isnull().sum().sum()

0

## Longitud

In [48]:
data['lng'] = data['lng'].fillna(method = "ffill")

In [49]:
data["lng"].isnull().sum().sum()

2

In [50]:
data['lng'] = data['lng'].fillna(method = "bfill")

In [51]:
data["lng"].isnull().sum().sum()

0

In [52]:
test['lng'] = test['lng'].fillna(method = "ffill")

In [53]:
test["lng"].isnull().sum().sum()

0

In [54]:
len(test)

60000

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [55]:
train_set = pd.DataFrame()

In [56]:
test_set = pd.DataFrame()

Primero de todo, se agrega el id y el precio asocidado. A partir de esto, se agregaran los features que se consideren necesarios.

In [57]:
train_set["id"] = data["id"]
train_set["precio"] = data["precio"]

In [58]:
train_set.head()

,id,precio
11188,192202,315000.0
52476,94838,2500000.0
30789,7819,1315600.0
13057,62385,8500000.0
138118,284662,680000.0


A partir de esto se generan features.

Igual pero para test_set. Solo que sin el precio.

In [59]:
test_set["id"] = test["id"]

In [60]:
test_set.head()

,id
58125,47104
14468,175113
14055,253170
4218,222487
56787,45818


In [61]:
len(test_set)

60000

# Se codifican las variables categoricas

In [62]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [63]:
train_set['train'] = True
test_set["train"] = False
combined_set = pd.concat([train_set, test_set], sort = False)

In [64]:
len(combined_set)

300000

In [65]:
combined['tipodepropiedad'].nunique()

24

In [66]:
combined['ciudad'].nunique()

921

Ciudad tiene demasiados valores posibles para OneHotEncoding

In [67]:
combined['provincia'].nunique()

32

In [68]:
combined_set['tipodepropiedad'] = combined['tipodepropiedad']
combined_set['provincia'] = combined['provincia']

In [69]:
len(combined_set)

300000

In [70]:
var_categoricas = ['tipodepropiedad', 'provincia']

In [71]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined_set[var_categoricas])
one_hot_encoded.columns

Index(['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_23',
  

In [72]:
combined_set = combined_set.join(one_hot_encoded)
combined_set.columns

Index(['id', 'precio', 'train', 'tipodepropiedad', 'provincia',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'p

In [73]:
len(combined_set)

420000

In [74]:
len(train_set)

240000

In [75]:
len(test_set)

60000

In [76]:
train_set = combined_set[combined_set["train"] == True]
test_set = combined_set[combined_set["train"] == False]

In [77]:
len(train_set)

300000

In [78]:
len(test_set)

120000

In [79]:
train_set = train_set.drop(columns = ['tipodepropiedad', 'provincia' , 'train'])

In [80]:
train_set.columns

Index(['id', 'precio', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_

In [81]:
test_set = test_set.drop(columns = ['tipodepropiedad', 'precio' , 'provincia', 'train'])

In [82]:
test_set.columns

Index(['id', 'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_2

In [83]:
len(test)

60000

In [84]:
len(test_set)

120000

In [85]:
test_set = test_set.drop_duplicates("id")

In [86]:
len(test_set)

60000

In [87]:
train_set = train_set.drop_duplicates("id")

In [88]:
len(train_set)

240000

## Se agregan todas las columnas numericas que ya se tienen

In [89]:
data.dtypes

id                               int32
titulo                          object
descripcion                     object
tipodepropiedad               category
direccion                       object
ciudad                          object
provincia                     category
antiguedad                     float64
habitaciones                   float64
garages                        float64
banos                          float64
metroscubiertos                float64
metrostotales                  float64
idzona                          object
lat                            float64
lng                            float64
fecha                           object
gimnasio                       float64
usosmultiples                  float64
piscina                        float64
escuelascercanas               float64
centroscomercialescercanos     float64
precio                         float64
train                             bool
dtype: object

Agrego los valores numericos que no poseen NULLs.

In [90]:
data_num = data.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [91]:
train_set = train_set.merge(data_num, on = ["id"], how = "inner")
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,254099,2273000.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,53461,3600000.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
2,247984,1200000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,209067,650000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
4,185997,1150000.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [92]:
test_num = test.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [93]:
test_set = test_set.merge(test_num, on = ["id"], how = "inner")
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,51775,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,115253,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
3,299321,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,173570,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0


## Se agregan las variables que fueron filleadas

In [94]:
data_num_fill = data.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [95]:
train_set = train_set.merge(data_num_fill, on = 'id', how = 'inner')
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,254099,2273000.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,20.0,80.0,80.0,1.0,2.0,19.393583,-99.159777
1,53461,3600000.0,0,0,0,0,0,1,0,0,...,0.0,1.0,1.0,10.0,180.0,268.0,2.0,2.0,19.310205,-99.227655
2,247984,1200000.0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,5.0,166.0,144.0,2.0,2.0,20.662590,-103.212280
3,209067,650000.0,0,1,0,0,0,0,0,0,...,0.0,1.0,1.0,1.0,67.0,63.0,1.0,1.0,19.301890,-99.688015
4,185997,1150000.0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,10.0,95.0,95.0,1.0,1.0,20.631040,-103.443312


In [96]:
test_num_fill = test.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [97]:
test_set = test_set.merge(test_num_fill, on = 'id', how = 'inner')
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,4941,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,29.0,112.0,300.0,1.0,4.0,19.408668,-99.246767
1,51775,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,67.0,67.0,1.0,1.0,21.032480,-89.592424
2,115253,0,1,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,100.0,87.0,1.0,2.0,19.332829,-99.152913
3,299321,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,2.0,86.0,86.0,2.0,2.0,16.860487,-99.878383
4,173570,0,0,0,1,0,0,0,0,0,...,0.0,1.0,1.0,10.0,76.0,80.0,1.0,1.0,19.640482,-99.127273


In [98]:
len(test_set)

60000

# Nuevos features

# Promedio de metroscubiertos por tipo de propiedad

In [99]:
metros_mean = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [100]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,117.049615
1,Bodega comercial,250.651494
2,Casa,194.067338
3,Casa en condominio,187.819091
4,Casa uso de suelo,253.124294


In [101]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,117.049615
1,Bodega comercial,250.651494
2,Casa,194.067338
3,Casa en condominio,187.819091
4,Casa uso de suelo,253.124294


In [102]:
data = data.merge(metros_mean, on = 'tipodepropiedad')

In [103]:
train_set = train_set.merge(data[['id', 'metroscubiertos_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [104]:
metros_mean = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [105]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,116.177500
1,Bodega comercial,250.569444
2,Casa,193.996470
3,Casa en condominio,186.668296
4,Casa uso de suelo,242.400901


In [106]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,116.177500
1,Bodega comercial,250.569444
2,Casa,193.996470
3,Casa en condominio,186.668296
4,Casa uso de suelo,242.400901


In [107]:
test = test.merge(metros_mean, on = 'tipodepropiedad')

In [108]:
test_set = test_set.merge(test[['id', 'metroscubiertos_mean_tipodeprop']], on = "id", how = "left")

In [109]:
len(test_set)

60000

In [110]:
test_set.fillna(method = "ffill", inplace = True)

# Varianza de metroscubiertos por tipo de propiedad

In [111]:
metros_var = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [112]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)
metros_var.fillna(method = "ffill", inplace = True)

In [113]:
data = data.merge(metros_var, on = 'tipodepropiedad')

In [114]:
train_set = train_set.merge(data[['id', 'metroscubiertos_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [115]:
metros_var = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [116]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)
metros_var.fillna(method = "ffill", inplace = True)

In [117]:
test = test.merge(metros_var, on = 'tipodepropiedad')

In [118]:
test_set = test_set.merge(test[['id', 'metroscubiertos_var_tipodeprop']], on = "id", how = "left")

In [119]:
len(test_set)

60000

In [120]:
test_set.fillna(method = "ffill", inplace = True)

# Se exporta el train_set final para uso de los modelos

## Suma de nulls totales. TIENE que ser 0. Sino el modelo no corre

In [121]:
train_set.isnull().sum().sum()

0

In [122]:
train_set.to_csv("train_set.csv", index = False)

In [123]:
test_set.isnull().sum().sum()

0

In [124]:
len(test_set)

60000

In [125]:
test_set.to_csv("test_set.csv", index = False)